In [14]:
%load_ext autoreload
%autoreload 2

In [15]:
import branca
import folium
import geojson
import geopandas as gpd
import pandas as pd
import requests

In [16]:
accessibility = requests.get('https://sidewalk-amsterdam.cs.washington.edu/v2/access/score/streets?lng1=4.769276&lat1=52.300761&lng2=5.004452&lat2=52.426341').content

accessibility_geojson = geojson.loads(accessibility)
accessibility_geojson

{"features": [{"geometry": {"coordinates": [[4.948992, 52.401247], [4.948996, 52.401417]], "type": "LineString"}, "properties": {"feature": {"CurbRamp": 9.0, "NoCurbRamp": 0.0, "Obstacle": 2.0, "SurfaceProblem": 0.0}, "osm_id": 7041317, "score": 0.991422514586288, "significance": {"CurbRamp": 0.75, "NoCurbRamp": -1.0, "Obstacle": -1.0, "SurfaceProblem": -1.0}, "street_edge_id": 1617}, "type": "Feature"}, {"geometry": {"coordinates": [[4.947361, 52.40102], [4.947441, 52.401021], [4.947487, 52.401018], [4.947541, 52.401009], [4.947596, 52.400991], [4.94831, 52.400702], [4.948349, 52.400694], [4.948373, 52.400695]], "type": "LineString"}, "properties": {"feature": {"CurbRamp": 1.0, "NoCurbRamp": 5.0, "Obstacle": 1.0, "SurfaceProblem": 0.0}, "osm_id": 7041323, "score": 0.005220125693558397, "significance": {"CurbRamp": 0.75, "NoCurbRamp": -1.0, "Obstacle": -1.0, "SurfaceProblem": -1.0}, "street_edge_id": 1619}, "type": "Feature"}, {"geometry": {"coordinates": [[4.944573, 52.40262], [4.9443

In [17]:
accessibility_gdf = gpd.GeoDataFrame.from_features(accessibility_geojson['features'])
accessibility_gdf

,geometry,street_edge_id,osm_id,score,significance,feature
0,"LINESTRING (4.94899 52.40125, 4.94900 52.40142)",1617,7041317,0.991423,"{'CurbRamp': 0.75, 'NoCurbRamp': -1.0, 'Obstac...","{'CurbRamp': 9.0, 'NoCurbRamp': 0.0, 'Obstacle..."
1,"LINESTRING (4.94736 52.40102, 4.94744 52.40102...",1619,7041323,0.005220,"{'CurbRamp': 0.75, 'NoCurbRamp': -1.0, 'Obstac...","{'CurbRamp': 1.0, 'NoCurbRamp': 5.0, 'Obstacle..."
2,"LINESTRING (4.94457 52.40262, 4.94437 52.40243)",1621,7041326,0.268941,"{'CurbRamp': 0.75, 'NoCurbRamp': -1.0, 'Obstac...","{'CurbRamp': 0.0, 'NoCurbRamp': 0.0, 'Obstacle..."
3,"LINESTRING (4.94572 52.40254, 4.94554 52.40256...",1622,7041326,0.731059,"{'CurbRamp': 0.75, 'NoCurbRamp': -1.0, 'Obstac...","{'CurbRamp': 8.0, 'NoCurbRamp': 0.0, 'Obstacle..."
4,"LINESTRING (4.94532 52.40292, 4.94497 52.40293...",1623,7041326,0.119203,"{'CurbRamp': 0.75, 'NoCurbRamp': -1.0, 'Obstac...","{'CurbRamp': 4.0, 'NoCurbRamp': 2.0, 'Obstacle..."
...,...,...,...,...,...,...
1638,"LINESTRING (4.96026 52.30967, 4.96002 52.30959)",34085,1006194153,0.500000,"{'CurbRamp': 0.75, 'NoCurbRamp': -1.0, 'Obstac...","{'CurbRamp': 0.0, 'NoCurbRamp': 0.0, 'Obstacle..."
1639,"LINESTRING (4.96080 52.30986, 4.96026 52.30967)",34086,1006194154,0.500000,"{'CurbRamp': 0.75, 'NoCurbRamp': -1.0, 'Obstac...","{'CurbRamp': 0.0, 'NoCurbRamp': 0.0, 'Obstacle..."
1640,"LINESTRING (4.96031 52.30960, 4.96073 52.30974...",34087,1006194155,0.500000,"{'CurbRamp': 0.75, 'NoCurbRamp': -1.0, 'Obstac...","{'CurbRamp': 0.0, 'NoCurbRamp': 0.0, 'Obstacle..."
1641,"LINESTRING (4.96008 52.30952, 4.96031 52.30960)",34088,1006194156,0.500000,"{'CurbRamp': 0.75, 'NoCurbRamp': -1.0, 'Obstac...","{'CurbRamp': 0.0, 'NoCurbRamp': 0.0, 'Obstacle..."


In [19]:
def calculate_accessibility_level(score):
    if score < 0.25:
        return 0
    elif score < 0.5:
        return 1
    elif score < 0.75:
        return 2
    else:
        return 3


accessibility_gdf['score_0to3'] = accessibility_gdf['score'].apply(calculate_accessibility_level)
accessibility_gdf

,geometry,street_edge_id,osm_id,score,significance,feature,score_0to3
0,"LINESTRING (4.94899 52.40125, 4.94900 52.40142)",1617,7041317,0.991423,"{'CurbRamp': 0.75, 'NoCurbRamp': -1.0, 'Obstac...","{'CurbRamp': 9.0, 'NoCurbRamp': 0.0, 'Obstacle...",3
1,"LINESTRING (4.94736 52.40102, 4.94744 52.40102...",1619,7041323,0.005220,"{'CurbRamp': 0.75, 'NoCurbRamp': -1.0, 'Obstac...","{'CurbRamp': 1.0, 'NoCurbRamp': 5.0, 'Obstacle...",0
2,"LINESTRING (4.94457 52.40262, 4.94437 52.40243)",1621,7041326,0.268941,"{'CurbRamp': 0.75, 'NoCurbRamp': -1.0, 'Obstac...","{'CurbRamp': 0.0, 'NoCurbRamp': 0.0, 'Obstacle...",1
3,"LINESTRING (4.94572 52.40254, 4.94554 52.40256...",1622,7041326,0.731059,"{'CurbRamp': 0.75, 'NoCurbRamp': -1.0, 'Obstac...","{'CurbRamp': 8.0, 'NoCurbRamp': 0.0, 'Obstacle...",2
4,"LINESTRING (4.94532 52.40292, 4.94497 52.40293...",1623,7041326,0.119203,"{'CurbRamp': 0.75, 'NoCurbRamp': -1.0, 'Obstac...","{'CurbRamp': 4.0, 'NoCurbRamp': 2.0, 'Obstacle...",0
...,...,...,...,...,...,...,...
1638,"LINESTRING (4.96026 52.30967, 4.96002 52.30959)",34085,1006194153,0.500000,"{'CurbRamp': 0.75, 'NoCurbRamp': -1.0, 'Obstac...","{'CurbRamp': 0.0, 'NoCurbRamp': 0.0, 'Obstacle...",2
1639,"LINESTRING (4.96080 52.30986, 4.96026 52.30967)",34086,1006194154,0.500000,"{'CurbRamp': 0.75, 'NoCurbRamp': -1.0, 'Obstac...","{'CurbRamp': 0.0, 'NoCurbRamp': 0.0, 'Obstacle...",2
1640,"LINESTRING (4.96031 52.30960, 4.96073 52.30974...",34087,1006194155,0.500000,"{'CurbRamp': 0.75, 'NoCurbRamp': -1.0, 'Obstac...","{'CurbRamp': 0.0, 'NoCurbRamp': 0.0, 'Obstacle...",2
1641,"LINESTRING (4.96008 52.30952, 4.96031 52.30960)",34088,1006194156,0.500000,"{'CurbRamp': 0.75, 'NoCurbRamp': -1.0, 'Obstac...","{'CurbRamp': 0.0, 'NoCurbRamp': 0.0, 'Obstacle...",2


In [20]:
m = folium.Map(
    location=[52.3676, 4.9041],
    tiles="cartodbpositron",
    zoom_start=13
)

In [21]:
colorscale = branca.colormap.step.RdYlGn_04.scale(0, 3)

accessibility_gdf.crs = 'urn:ogc:def:crs:OGC:1.3:CRS84'
folium.GeoJson(accessibility_gdf, name="accessibility",
               style_function=lambda feature: {'color': colorscale(feature['properties']['score_0to3'])}).add_to(m)
folium.LayerControl().add_to(m)
m